In [1]:
import torch
import cv2
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%run helpers.ipynb
%run NN.ipynb
np.set_printoptions(precision=2)
torch.manual_seed(42)
np.random.seed(42)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# Hyper parameters

In [3]:
canvas_width = 51
canvas_height= 51
center = np.array([(canvas_height-1) / 2, (canvas_width-1) / 2])
num_drawings = 100
num_samples = 200
bins = np.arange(0,21)

ts = np.array([0.17,0.2,0.08,0.1])

# Generate the dataset

In [4]:
canvass_train = np.zeros([num_drawings,canvas_height, canvas_width])
canvass_test = np.zeros([num_drawings,canvas_height, canvas_width])

In [5]:
for i in range(num_drawings):
    if np.random.rand() > 0.5:
        canvass_train[i] = draw_polygon(canvass_train[2],3,5)
    else:
        canvass_train[i] = draw_ellipse(canvass_train[i],10,15)

for i in range(num_drawings):
    if np.random.rand() > 0.5:
        canvass_test[i] = draw_polygon(canvass_test[2],3,5)
    else:
        canvass_test[i] = draw_ellipse(canvass_test[i],10,15)

# Get the length vectors

In [6]:
lengthss_train = get_length_encoding(canvass_train,num_samples,bins)
lengthss_test = get_length_encoding(canvass_test,num_samples,bins)

# Turn image into graph

In [7]:
lap_mats_train = adj_mat_to_lap_mat(imgs_to_adj_mat(canvass_train))
lap_mats_test = adj_mat_to_lap_mat(imgs_to_adj_mat(canvass_test))

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


# Compute eigenvalues & eigenvectors &hks

In [8]:
evals_train,Us_train = lap_mat_to_eigen(lap_mats_train)
hkss_train = batch_heat_kernel_signature(ts, evals_train, Us_train)

evals_test,Us_test = lap_mat_to_eigen(lap_mats_test)
hkss_test = batch_heat_kernel_signature(ts, evals_test, Us_test)

# Set up Neural Network

In [9]:
class HeatKernelSignatureNet(nn.Module):
    #takes in image (51 x 51), returns heat kernel signature at [0.17,0.47,0.58,0.81]

    def __init__(self,image_width,image_height):
        super().__init__()
        self.image_width = image_width
        self.image_height = image_height

        self.pool = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

        self.MLP = MLP(48,4,10,100)


    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        x = self.pool(torch.tanh(self.conv3(x)))

        x = torch.flatten(x,start_dim=1,end_dim=-1)
        x = self.MLP(x)

        return x

In [10]:
hksNet = HeatKernelSignatureNet(51,51)
hksNet.to(device)

HeatKernelSignatureNet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (MLP): MLP(
    (model): Sequential(
      (fc1): Linear(in_features=48, out_features=100, bias=True)
      (tanh1): Tanh()
      (fc2): Linear(in_features=100, out_features=100, bias=True)
      (tanh2): Tanh()
      (fc3): Linear(in_features=100, out_features=100, bias=True)
      (tanh3): Tanh()
      (fc4): Linear(in_features=100, out_features=100, bias=True)
      (tanh4): Tanh()
      (fc5): Linear(in_features=100, out_features=100, bias=True)
      (tanh5): Tanh()
      (fc6): Linear(in_features=100, out_features=100, bias=True)
      (tanh6): Tanh()
      (fc7): Linear(in_features=100, out_features=100, bias=True)
      (tanh7): Tanh()
      (fc8): Linear(in_features=100, out_features=100, bias=

In [11]:
canvass_train_torch = torch.tensor(canvass_train).to(device).float()
canvass_train_torch = torch.reshape(canvass_train_torch,[num_drawings,1,canvas_width,canvas_height])

canvass_test_torch = torch.tensor(canvass_test).to(device).float()
canvass_test_torch = torch.reshape(canvass_test_torch,[num_drawings,1,canvas_width,canvas_height])

hkss_train_torch = torch.tensor(hkss_train).to(device).float()
hkss_test_torch = torch.tensor(hkss_test).to(device).float()

In [12]:
learning_rates = [0.005] * 2000 + [0.0005] * 2000 + [0.00005] * 2000

In [13]:
for i in range(len(learning_rates)):
    opt = torch.optim.AdamW(list(hksNet.parameters()),learning_rates[i])
    opt.zero_grad()

    pred_hks = hksNet(canvass_train_torch)
    loss = torch.mean(( (pred_hks - hkss_train_torch) / hkss_train_torch )**2)
    loss.backward()
    opt.step()

    if i % 10 == 0:
        print(i,loss.item())

0 1.0577194690704346
10 0.11155016720294952
20 0.10844852775335312
30 0.11175420135259628
40 0.12559448182582855
50 0.13086269795894623
60 0.14109867811203003
70 0.15023352205753326
80 0.10464713722467422
90 0.08999171853065491
100 0.11308469623327255
110 0.0793231800198555
120 0.01574951782822609
130 0.16162581741809845
140 0.1151495948433876
150 0.029542239382863045
160 0.009614511393010616
170 0.015491340309381485
180 0.016168462112545967
190 0.02346944622695446
200 0.06915082782506943
210 0.22496938705444336
220 0.13043813407421112
230 0.05652587115764618
240 0.1345088928937912
250 0.10228119045495987
260 0.09078903496265411
270 0.07976115494966507
280 0.13409122824668884
290 0.05820099264383316
300 0.060952797532081604
310 0.06968194991350174
320 0.046608228236436844
330 0.03725931793451309
340 0.0429488867521286
350 0.03518855571746826
360 0.033112019300460815
370 0.02540845237672329
380 0.06433960795402527
390 0.07196997851133347
400 0.05321565642952919
410 0.028080826625227928


KeyboardInterrupt: 

# Test

In [14]:
pred_hks_test = hksNet(canvass_test_torch)
loss_test = torch.mean(( ( pred_hks_test - hkss_test_torch) / hkss_test_torch )**2)
print(loss_test)

tensor(0.0026, grad_fn=<MeanBackward0>)


# Randomly sample a few to see what it looks like

In [15]:
print(pred_hks_test[15])
print(hkss_test_torch[15])

tensor([3.8529, 4.9785, 1.8480, 2.1630], grad_fn=<SelectBackward0>)
tensor([3.8336, 4.9593, 1.8316, 2.1480])


In [16]:
print(pred_hks_test[35])
print(hkss_test_torch[35])

tensor([3.9131, 5.0612, 1.8715, 2.1914], grad_fn=<SelectBackward0>)
tensor([3.8539, 4.9875, 1.8376, 2.1563])


In [17]:
print(pred_hks_test[60])
print(hkss_test_torch[60])

tensor([3.8326, 4.9901, 1.8483, 2.1524], grad_fn=<SelectBackward0>)
tensor([3.7467, 4.8356, 1.8077, 2.1144])


In [18]:
print(pred_hks_test[96])
print(hkss_test_torch[96])

tensor([3.8649, 5.0191, 1.8528, 2.1645], grad_fn=<SelectBackward0>)
tensor([3.7738, 4.8761, 1.8140, 2.1236])
